Imports

In [6]:
from plantfusion.lgrass_wrapper import Simpraise_wrapper # L-grass FSPM management : TO BE IMPLEMENTED
# see : https://github.com/mwoussen/plantfusion/blob/develop/simulations/other_fspm.py
from plantfusion.light_wrapper import Light_wrapper # light management using LightVegeManager
from plantfusion.soil_wrapper import Soil_wrapper # soil management
from plantfusion.indexer import Indexer # index management
from plantfusion.planter import Planter # plant position management

import time
import datetime
import os

Path management

In [7]:
in_folder_lgrass = "inputs_lgrass/inputs"
genet_folder_lgrass = "inputs_lgrass/modelgenet"
out_folder = "outputs/lgrass"

id1 = 17111
id2 = 17112
writegeo = True

try:
    # Create target Directory
    os.mkdir(os.path.normpath(out_folder))
    print("Directory ", os.path.normpath(out_folder), " Created ")
except FileExistsError:
    print("Directory ", os.path.normpath(out_folder), " already exists")


Directory  outputs\lgrass  already exists


Already implemented in plantfusion (should run if given appropriate parameters)

In [8]:
lgrass_name = "lgrass"

indexer = Indexer(global_order=[lgrass_name], other_names=[lgrass_name])

generation_type = "random"
plant_density = {lgrass_name : 450} # plantes.m-2
planter = Planter(generation_type=generation_type, indexer=indexer, plant_density=plant_density)
#note : arg "inter_rows" is not needed for random generation (I think), but is set to 0.15 anyway.

sky = "turtle46"
lighting_caribu = Light_wrapper(
        lightmodel="caribu",
        indexer=indexer, 
        planter=planter, 
        sky=sky,
        writegeo=False,
    )


soil = Soil_wrapper(
    in_folder='inputs_soil_legume', 
    out_folder=out_folder, 
    IDusm=1711, # unit of simulation
    planter=planter, 
    opt_residu=0, 
    save_results=True
)


Directory  outputs\lgrass\soil  already exists


Lgrass Wrapper instanciation

In [9]:
print(os.getcwd())
lgrass = Simpraise_wrapper(
    name=lgrass_name,
    indexer=indexer,
    in_folder=in_folder_lgrass, genet_folder=genet_folder_lgrass, out_folder=out_folder,
    planter=planter,
    plan_sim_file='plan_simulation.csv',
    genet_file='ped.r',
    param_plant_file='liste_plantes.csv'
    #ADD ARGS SPECIFIC TO L-GRASS
)

c:\Users\u242825\Desktop\Gembloux\Recherche\plant_modelling\lgrass_legume_fusion\plantfusion
0       LUSIGNAN
1       LUSIGNAN
2       LUSIGNAN
3       LUSIGNAN
4       LUSIGNAN
          ...   
7443    LUSIGNAN
7444    LUSIGNAN
7445    LUSIGNAN
7446    LUSIGNAN
7447    LUSIGNAN
Name: site, Length: 7448, dtype: object
treatment_1


<string>:1400: Warning: Found symbol '#' after Lstring. Considered as begining of comments
lgrass.lpy:1597: Warning: IndentationWarning: missing indent.
lgrass.lpy:1612: Warning: IndentationWarning: missing indent.
lgrass.lpy:1728: Warning: IndentationWarning: missing indent.
c:\users\u242825\desktop\gembloux\recherche\plant_modelling\lgrass_legume_fusion\plantfusion\lgrass\lgrass\param_reproduction_functions.py:140: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform

Simulation Lgrass alone

In [11]:
try:
    current_time_of_the_system = time.time()
    for t in range(lgrass.lsystem.derivationLength):
        lgrass.derive(t)

        ### CARIBU
        scene_lgrass = lgrass.light_inputs(elements="triangles")
        start = time.time()
        lighting_caribu.run(
            scenes=[scene_lgrass], day=lgrass.lsystem.current_day, parunit="RG"
        )
        caribu_time = time.time() - start
        lgrass.light_results(legume.energy(), lighting_caribu)

        (
            N_content_roots_per_plant,
            roots_length_per_plant_per_soil_layer,
            plants_soil_parameters,
            plants_light_interception,
        ) = lgrass.soil_inputs()
        soil.run(
            lgrass.doy(),
            [N_content_roots_per_plant],
            [roots_length_per_plant_per_soil_layer],
            [plants_soil_parameters],
            [plants_light_interception],
        )
        lgrass.soil_results(soil.results, planter)

        lgrass.run()

        print("Lighting running time | CARIBU: ", caribu_time)

    execution_time = int(time.time() - current_time_of_the_system)
    print("\n" "Simulation run in {}".format(str(datetime.timedelta(seconds=execution_time))))

finally:
    lgrass.end()

0       LUSIGNAN
1       LUSIGNAN
2       LUSIGNAN
3       LUSIGNAN
4       LUSIGNAN
          ...   
7443    LUSIGNAN
7444    LUSIGNAN
7445    LUSIGNAN
7446    LUSIGNAN
7447    LUSIGNAN
Name: site, Length: 7448, dtype: object
LUSIGNAN
TPS : 1.0 Current day : 1


AttributeError: lstring

In [ ]:


current_time_of_the_system = time.time()

for t in range(legume.lsystem.derivationLength) :# chose derivation lengh

    legume.derive(t)

    # while lgrass.lsystem.current_day > t:
    print(lgrass.lstring)
    lgrass.derive(t)

    scene_legume = legume.light_inputs(elements="triangles")
    scene_lgrass = lgrass.light_inputs() # ARGS DEPENDS ON LGRASS CLASS IMPLEMENTATION
    scenes = indexer.light_scenes_mgmt({lgrass_name : scene_lgrass, legume_name : scene_legume})

    lighting.run(scenes=scenes, day=legume.doy(), parunit="RG") #what is parunit="RG" ?
    legume.light_results(legume.energy(), lighting)
    lgrass.light_results() # ARGS DEPENDS ON LGRASS CLASS IMPLEMENTATION

    soil_legume_inputs = legume.soil_inputs()
    soil_lgrass_inputs = lgrass.soil_inputs()

    (
        N_content_roots_per_plant,
        roots_length_per_plant_per_soil_layer,
        plants_soil_parameters,
        plants_light_interception, # what are these output's type ?
    ) = indexer.soil_inputs({legume_name : soil_legume_inputs, lgrass_name : soil_lgrass_inputs})

    soil.run( # update soil inputs
        legume.doy(), #doy : day of the year (julian day)
        N_content_roots_per_plant,
        roots_length_per_plant_per_soil_layer,
        plants_soil_parameters,
        plants_light_interception,
    )

    #compute new plant parameters for next derivation. To be implemented in Lgrass wrapper
    legume.run()
    lgrass.run()
execution_time = int(time.time() - current_time_of_the_system)
print("\n" "Simulation run in {}".format(str(datetime.timedelta(seconds=execution_time))))

# end() effect is specific to the model
legume.end()
lgrass.end()
soil.end()